# Evaluate Sensitivity of VCR threshold to LOS distribution

In [1]:
# import libraries
import sys
import warnings
import os
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime


sys.path.append('../../utils')
import data_paths
from traffic_counts import TrafficCounts
from hbefa_hot_emissions import HbefaHotEmissions
from calculate_vkt import calculate_VKT

warnings.filterwarnings("ignore")

# Reload local modules on changes
%reload_ext autoreload
%autoreload 2

## Notebook settings

In [2]:
# Define start and end time for emission calculation. Ideally this should cover a whole year.
start_date = datetime(2019, 1, 1)
end_date = datetime(2019, 12, 31)

# define filename of the visum file
visum_filename = "visum_links.GPKG"

# if True, the script will only calculate the emission for the area within the roi polygon
clip_to_area = True
roi_polygon = data_paths.MUNICH_BOARDERS_FILE # defines ROI for clipping

## Import Data

In [3]:
# import visum model, clip it to the ROI, and initialize cycles object
visum = gpd.read_file(data_paths.VISUM_FOLDER_PATH + visum_filename)

if clip_to_area:
    roi = gpd.read_file(roi_polygon).to_crs(visum.crs)
    visum = gpd.clip(visum, roi)
    visum = visum.explode(ignore_index=True) # convert multipolygons to polygons

#visum = visum_links
visum = visum.reset_index(drop = True).reset_index() # reset index for calculation
visum['road_lenght'] = visum['geometry'].length # add road length to visum dict

# initialize traffic cycles
cycles = TrafficCounts()
# initialize HBEFA emission factors
hbefa = HbefaHotEmissions()

Loaded emission factors from /Users/daniel_tum/Documents/code/drive-inventory/data/restricted_input/hbefa/EFA_HOT_ts_hbefa.txt
Loaded emission factors from /Users/daniel_tum/Documents/code/drive-inventory/data/restricted_input/hbefa/EFA_HOT_aggregated_hbefa.txt


In [13]:
perturbation_factor = 1.1

# define hbefa service thresholds
default_vcr = hbefa.default_vcr_thresholds
changed_vcr = {k: [round(v * perturbation_factor, 5) for v in vals] for k, vals in default_vcr.items()}

# apply service thresholds as defined in the notebook setting
hbefa.vcr_thresholds = changed_vcr

## Calculate VKT and traffic situation

In [14]:
# Calculate VKT per year and traffic condition for each vehicle class

vkt_result = dict()

# create list of dates
dates = [d.strftime("%Y-%m-%d") for d in pd.date_range(start=start_date, end=end_date, freq='1d')]
# extract list of years in dates list
years = list(set([d.split('-')[0] for d in dates]))
years.sort()

# sort dates by year
for year in years:
    # create sublist of dates for each year
    dates_in_year = [d for d in dates if d.startswith(year)]

    final_result = {'Freeflow': np.array(5, float),
                    'Heavy': np.array(5, float),
                    'Satur.': np.array(5, float),
                    'St+Go': np.array(5, float), 
                    'St+Go2': np.array(5, float)}
    
    # calculate for each date in the year
    for date in dates_in_year: 
        cl, vehicle_index = calculate_VKT(date = date,
                                          visum_dict = visum.to_dict('records'),
                                          cycles_obj = cycles,
                                          hbefa_obj = hbefa)
        
        for key, value in cl.items():
            final_result[key] = final_result[key] + value
    vkt_result.update({year:final_result})

In [15]:
# calculate LOS share for the respective year
los_class_vkt = pd.DataFrame()
for key, item in vkt_result.items(): 
    df = pd.DataFrame(data=item, index = vehicle_index).sum(axis = 0)
    df.name = key
    los_class_vkt = pd.concat([los_class_vkt, df], axis = 1)

los_class_vkt = los_class_vkt.transpose() 
los_class_vkt = los_class_vkt.divide(los_class_vkt.sum(axis=1), axis =0)
los_class_vkt

,Freeflow,Heavy,Satur.,St+Go,St+Go2
2019,0.600648,0.220443,0.142288,0.026778,0.009842


In [16]:
#vkt_2019_nominal = los_class_vkt
#vkt_2019_minus10 = los_class_vkt
vkt_2019_plus10 = los_class_vkt

In [17]:
vkt_final = pd.concat([vkt_2019_nominal, vkt_2019_minus10, vkt_2019_plus10])

In [18]:
vkt_final

,Freeflow,Heavy,Satur.,St+Go,St+Go2
2019,0.533783,0.225212,0.162909,0.055798,0.022297
2019,0.465626,0.216708,0.175684,0.082419,0.059563
2019,0.600648,0.220443,0.142288,0.026778,0.009842
